In [11]:
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
import gspread
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from apiclient import discovery
from apiclient.http import MediaFileUpload
# from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import googleapiclient.discovery

import sa_drive as drive

In [12]:
files = drive.get_sibs_files()
print(len(files))
print(files['name'])

Drives:
Helpo (0AG3pjV8_oFy2Uk9PVA)
100


TypeError: list indices must be integers or slices, not str

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('SA_sheet-write-access.json', scope)
client = gspread.authorize(creds)

sheet = client.open('Helpo-SIBS').sheet1

data = sheet.get_all_records()
sheet_df = pd.DataFrame(data)
sheet_df.head()

,test1,tes2,test3
0,1,2,3
1,11,22,33
2,111,222,333


In [271]:
widths = [1, 4, 7, 7, 9, 9, 5, 3, 2, 73]
header = ['reg', 'type', 'IDori', 'IDdest', 'lastDate', 'IDlast', 'entity', 'coin', 'VAT', 'filler']
file_info = pd.read_fwf('202002040340.inp', widths=widths, nrows=1, header=None) 
file_info.columns = header
file_info.head()


,reg,type,IDori,IDdest,lastDate,IDlast,entity,coin,VAT,filler
0,0,MSTM,1,334221,202002041,202002031,10004,978,23,NaN


In [253]:
widths = [1,2,1,4,8,12,1,10,5,15,10,5, 9, 9, 8, 1, 1, 12, 6]
header = ['reg', 'cod', 'PDD', 'IdSIBS', 'numSIBS','date', 'typeT', 'idT', 'idTrans', 'LocT', 'value', 'tarif', 'refMB', 'NIF', 'nFact', 'modCom', 'CodEmp', 'RespID', 'filler']
df = pd.read_fwf('2020020403dsdfd40.inp', widths=widths, skipfooter=1) 
df.columns = header


In [254]:
df['date']= pd.to_duu

In [258]:
df.head()

,reg,cod,PDD,IdSIBS,numSIBS,date,typeT,idT,idTrans,LocT,value,tarif,refMB,NIF,nFact,modCom,CodEmp,RespID,filler
0,1,4,2,460,363385,2020-02-04 08:37:00,P,0,0,MBWAY,200,0,111111111,240571215,41,0,0,NaN,NaN
1,1,4,2,461,663982,2020-02-04 15:53:00,P,0,0,MBWAY,1000,0,111111111,0,42,0,0,NaN,NaN


In [259]:
excelFrame = df[['date', 'NIF', 'value', 'LocT']]

In [262]:
excelFrame.head()

,date,NIF,value,LocT
0,2020-02-04 08:37:00,240571215,200,MBWAY
1,2020-02-04 15:53:00,0,1000,MBWAY


In [264]:
writer = ExcelWriter('SIBS_exporter.xlsx')
excelFrame.to_excel(writer,'test')
writer.save()